In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from openpyxl import Workbook
import time

job_needed = input("Which job title are you looking for?")
location_needed = input("Which location would you prefer?")

s = Service('/Users/manit57/Untitled Folder/chromedriver')
driver = webdriver.Chrome(service=s)
driver.get("https://www.linkedin.com/jobs/search?trk=guest_homepage-basic_guest_nav_menu_jobs&position=1&pageNum=0")

search_job = driver.find_element(By.ID, 'job-search-bar-keywords')
search_location = driver.find_element(By.ID, 'job-search-bar-location')

search_job.send_keys(job_needed)
search_location.clear()
search_location.send_keys(location_needed + Keys.ENTER)

# Get the URL after the input
new_url = driver.current_url

# Wait for the page to load
time.sleep(2)

# Scroll down the page a few times to load more job listings
for _ in range(9):  # Scroll down 5 times, adjust as needed
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(3)  # Wait for the page to scroll and load more job listings

# Click "See more jobs" button
see_more_button = driver.find_element(By.CLASS_NAME, 'infinite-scroller__show-more-button')
see_more_button.click()


# Wait for the new job listings to load
time.sleep(5)

job_listings = driver.find_elements(By.CLASS_NAME, 'base-search-card__info')

# Create an Excel workbook and select the active worksheet
wb = Workbook()
ws = wb.active

# Add headers
ws.append(["Job Title", "Company Name", "Link"])

for listing in job_listings:
    job_title_element = listing.find_element(By.CLASS_NAME, 'base-search-card__title')
    job_title = job_title_element.text.strip()  # Strip any leading or trailing whitespace
    
    # Find company name
    info_div = listing.find_element(By.CLASS_NAME, 'base-search-card__subtitle')
    company_name = info_div.text.strip()  # Strip any leading or trailing whitespace
    
    # Find link
    link_element = listing.find_element(By.TAG_NAME, 'a')
    link = link_element.get_attribute('href')
    
    # Append job title, company name, and link to Excel sheet
    ws.append([job_title, company_name, link])

# Save the Excel file
wb.save("{0}_job_listings_{1}.xlsx".format(job_needed, location_needed))

# Quit the browser
driver.quit()
